In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import ast
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 2numGB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
df = pd.read_csv('../dune_tx/merged/total_merged.csv')
# the total number of rows in the dataset
print(df.shape[0])

2473502


In [12]:
df.sort_values('rule_name_1', inplace=True)
df.head()

,tx_hash,rule_id_1,similarity_1,rule_name_1,rule_id_2,similarity_2,rule_name_2
1917345,0x8cf5a2d4922f7e981760f197e99d7da7e877b6a9f3b0...,240,0.223529,0x0DEX,364,0.2,GFA
2195119,0x5b1a8f27d222c7ad7f2bafadaa5bcd358573d6c3a258...,240,0.223529,0x0DEX,364,0.2,GFA
2195122,0x0369f9125e8d4cae7c9b9d80fdb7a0bff0b0feb06063...,240,0.223529,0x0DEX,364,0.2,GFA
2195123,0x0ff0a8c17bd8b898353ec71f5ba13ca4daf04ec0a4d9...,240,0.223529,0x0DEX,364,0.2,GFA
1963895,0x67d38211087f4dd5f1dcf340275795a63248c2c2b03b...,240,0.223529,0x0DEX,364,0.2,GFA


In [13]:
df['group_count'] = df.groupby(['rule_id_1', 'similarity_1'])['tx_hash'].transform('count')
grouped = df.groupby(['rule_id_1', 'similarity_1'])
classified_list = []
unique_list = []

for _, group in grouped:
    if len(group) == 1:
        unique_list.append(group.iloc[0])  # 直接添加到单独表
    else:
        classified_list.append(group.iloc[0])  # 只保留每个分组的第一个记录

classified_df = pd.DataFrame(classified_list).reset_index(drop=True)
unique_df = pd.DataFrame(unique_list).reset_index(drop=True)

classified_df.insert(0, "ID", range(1, len(classified_df) + 1))
unique_df.insert(0, "ID", range(1, len(unique_df) + 1))

classified_df.rename(columns={'group_count': 'count'}, inplace=True)

In [14]:
classified_df = classified_df[classified_df['count'] < 150]
# create a new dataframe having to columns: tx_hash and label, all the value of label is 0
classified_df['attack'] = 0
classified_df = classified_df[['tx_hash', 'attack']]
classified_df


,tx_hash,attack
0,0x5d1f861bab04688fb5219de9502a16dde72f7d5c65e6...,0
1,0x1c55def937d185f5b50dfd501f204e5c229d09160d8b...,0
2,0x4e10bd3e86c9c48e38632c713ebf7e94628deec6cfa7...,0
3,0xca1fafc1f46795b0eb6e68c71313c328f44b395819fb...,0
4,0x20f8324eb357f288e439a829d068e8db22cf7211af1d...,0
...,...,...
8344,0x235ee0e0bee8fc99e1909c4e7609f2420d80b385e559...,0
8345,0x142bad59aa8fe7a8cda3e6f73b1766dde6ab0d8ae874...,0
8346,0xd0f50466583f5122dbc94b8aa20c8a84665b516b4be3...,0
8347,0xf2a3765cc8164d194a9512d423cdd237974a6f9a6223...,0


In [ ]:
# save the classified_df to csv file
classified_df.to_csv('benchmark-data-benigh.csv', index=False)

In [21]:
# calculate the sum of all the count
total_count = classified_df['count'].sum() + unique_df['group_count'].sum()

classified_df = classified_df[classified_df['count'] > 150]
classified_df = classified_df.sort_values(by=['similarity_1'], ascending=False).reset_index(drop=True)

classified_df.to_csv("classified_tx_filtered.csv", index=False)
unique_df.to_csv("unique_tx.csv", index=False)

total_count

benign_txs = classified_df['tx_hash']
benign_names = classified_df['rule_name_1'] + (classified_df['similarity_1'] * 100).astype(int).astype(str) + '_benign' + classified_df['ID'].astype(str)
benign_names

0        Swapos80_benign4265
1       Uranium77_benign2489
2         Miner75_benign1256
3      Paraswap71_benign3877
4           LPC71_benign5700
               ...          
473       TSURU23_benign7966
474        BNBX23_benign7899
475         GFA23_benign5112
476        LQDX23_benign6445
477      0x0DEX22_benign3626
Length: 478, dtype: object